#**Step 1: installazione delle librerie**

In [1]:
!pip install -q accelerate
!pip install peft
!pip install bitsandbytes
!pip install transformers
!pip install trl
!pip install huggingface_hub
!pip install accelerate
!pip install --upgrade torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00


#**Step 2: caricamento delle librerie**

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

import pandas as pd
from datasets import Dataset
import re
import random

#**Step 3: impostazione delle variabili**

In [3]:
percentuale_dataset = 1
dataset_type = ""

# The model that you want to train from the Hugging Face hub
model_name = "sapienzanlp/Minerva-3B-base-v1.0"

# Number of training epochs
num_train_epochs = 2

# Fine-tuned model name
#rand = random.randint(0, 10000)
#rand = 1824
new_model = f"minerva_3B_it_{num_train_epochs}epoch"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

#**Step 4: caricamento del dataset**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

# change this to match your path
data_path_models = "./drive/MyDrive/Colab Notebooks/WebNLG-IT/Esperimento 2b/Models"
data_path_data = "./drive/MyDrive/Colab Notebooks/WebNLG-IT/Esperimento 2b/Datasets"

Mounted at /content/drive


In [5]:
# carico il dataset
trainset = pd.read_csv(f'{data_path_data}/WebNLG-IT/train.csv')
devset = pd.read_csv(f'{data_path_data}/WebNLG-IT/dev.csv')

# ottengo un sottoinsieme del dataset di partenza
subset_trainset = trainset.sample(frac=percentuale_dataset, random_state=42)
subset_trainset.head()

,triple,sentence
13581,Michele_Marcolini club Vicenza_Calcio A.C._Lum...,Michele Marcolini ha giocato per il Vicenza Ca...
14695,San_Sebastián_de_los_Reyes country Spain Adolf...,L'aeroporto Adolfo Suarez Madrid-Barajas si tr...
9976,Marriott_International foundationPlace Washing...,Marriott International è un inquilino dell'AC ...
2847,(15788)_1993_SB discoverer Roque_de_los_Muchac...,(15788) 1993 SB è stata scoperta dall'Osservat...
23428,Andrew_Rayel associatedBand/associatedMusicalA...,Andrew Rayel è stato associato ai seguenti art...


In [6]:
len(subset_trainset)

35422

In [7]:
# ottengo un sottoinsieme del dataset di partenza
subset_devset = devset.sample(frac=percentuale_dataset, random_state=42)
subset_devset.head()

,triple,sentence
297,Adare_Manor country Republic_of_Ireland,Adare Manor si trova nella Repubblica d'Irlanda.
969,India leader T._S._Thakur Agra_Airport locatio...,L'aeroporto di Agra si trova in India e uno de...
2037,Alaa_Abdul-Zahra club Al-Khor_Sports_Club Alaa...,La squadra di Alaa Abdul-Zahra è l'Al Shorta S...
598,Mexico demonym Mexicans,I messicani sono persone provenienti dal Messico.
3411,Aleksey_Chirikov_(icebreaker) builder Finland ...,"Il rompighiaccio, Aleksey Chirikov, è stato co..."


In [8]:
llama_trainset = []
for index, row in subset_trainset.iterrows():
    triple = row['triple']
    sentence = row['sentence']

    llama_trainset.append(f'<s> [INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE] [ANW] {sentence} [/ANW] </s>')

llama_trainset = pd.DataFrame(llama_trainset, columns=['instr'])
llama_trainset.head()

,instr
0,<s> [INST] Given the following triples in (TRI...
1,<s> [INST] Given the following triples in (TRI...
2,<s> [INST] Given the following triples in (TRI...
3,<s> [INST] Given the following triples in (TRI...
4,<s> [INST] Given the following triples in (TRI...


In [9]:
llama_trainset = Dataset.from_pandas(llama_trainset)

In [10]:
llama_devset = []
for index, row in subset_devset.iterrows():
    triple = row['triple']
    sentence = row['sentence']

    llama_devset.append(f'<s> [INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE] [ANW] {sentence} [/ANW] </s>')

llama_devset = pd.DataFrame(llama_devset, columns=['instr'])
llama_devset.head()

,instr
0,<s> [INST] Given the following triples in (TRI...
1,<s> [INST] Given the following triples in (TRI...
2,<s> [INST] Given the following triples in (TRI...
3,<s> [INST] Given the following triples in (TRI...
4,<s> [INST] Given the following triples in (TRI...


In [11]:
llama_devset = Dataset.from_pandas(llama_devset)

#**Step 5: caricamento del modello**

In [12]:
from huggingface_hub import login
token = ""
login(token=token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [13]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

Your GPU supports bfloat16: accelerate training with bf16=True


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/959 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/795k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


#**Step 6: fine-tuning del modello**

In [14]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    evaluation_strategy="steps",  # Calcolo della validation loss ad ogni step
    save_strategy="steps",  # Checkpoint del modello ad ogni step
    save_total_limit=3,  # Limita il numero di checkpoint salvati
    load_best_model_at_end=True,  # Carica il modello migliore al termine dell'addestramento
    metric_for_best_model="eval_loss",  # Sceglie la metrica per determinare il miglior modello
    greater_is_better=False,  # Indica se un valore più alto della metrica è migliore o no
    eval_steps=500,  # Numero di passaggi prima di valutare il modello
    logging_steps= 500,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=llama_trainset,
    eval_dataset=llama_devset,
    peft_config=peft_config,
    dataset_text_field="instr",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/35422 [00:00<?, ? examples/s]

Map:   0%|          | 0/4464 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
500,1.453700,0.899607
1000,0.738600,0.704883
1500,0.615600,0.618251
2000,0.550400,0.564033
2500,0.504800,0.515295
3000,0.472800,0.492175
3500,0.452900,0.464162
4000,0.433300,0.448794
4500,0.417500,0.433820
5000,0.403800,0.417389


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=17712, training_loss=0.4070257494443154, metrics={'train_runtime': 19014.4944, 'train_samples_per_second': 3.726, 'train_steps_per_second': 0.931, 'total_flos': 1.7111471724546048e+17, 'train_loss': 0.4070257494443154, 'epoch': 2.0})

In [15]:
# Sostituisci "path_to_save" con il percorso desiderato nel tuo drive
trainer.save_model(f'{data_path_models}/{new_model}')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


#**Step 7: generazione frasi del modello creato**

In [ ]:
del model
del training_arguments
del trainer

In [16]:
# Empty VRAM
import gc
gc.collect()
gc.collect()

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, f'{data_path_models}/{new_model}')
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

def generate_text(triple, actual):
  max_length = len(f"<s>[INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE] [ANW] {actual} [/ANW]") * 1.2

  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=int(max_length), temperature=0.1)
  result = pipe(f"<s>[INST] Given the following triples in (TRIPLE), you have to generate the corresponding text in (ANW) [/INST] [TRIPLE] {triple} [/TRIPLE]")

  if result is not None and result and len(result) > 0 and 'generated_text' in result[0]:
      generated_text = result[0]['generated_text']

      # Utilizziamo una regex per estrarre il contenuto all'interno del primo tag [ANW] ... [/ANW]
      match = re.search(r'\[ANW\] (.*?) \[/ANW\]', generated_text)

      if match:
          content_within_anw_tag = match.group(1)
          text = content_within_anw_tag
      else:
          text = generated_text

  else:
      text = "Il risultato della pipeline è nullo."

  return text

In [18]:
percentuale_dataset = 1

In [19]:
testset = pd.read_csv(f'{data_path_data}/WebNLG-IT/test.csv')

# ottengo un sottoinsieme del testset di partenza
subset_testset = testset.sample(frac=percentuale_dataset, random_state=42)
print(len(subset_testset))

# test
X_test = subset_testset['triple'].values.tolist()
y_test = subset_testset['sentence'].values.tolist()

7305


In [24]:
subset_testset.head()

,triple,sentence
7061,11th_Mississippi_Infantry_Monument category Co...,"Un monumento all'11° Fanteria del Mississippi,..."
4553,Bananaman broadcastedBy BBC BBC city Broadcast...,Bill Oddie è protagonista di un programma dell...
6504,Karlsruhe postalCode 76131–76229,I codici postali di Karlsruhe sono 76131-76229.
1918,Turkish_people religion Irreligion,Alcuni turchi sono irreligiosi.
5227,ENAIRE city Madrid Adolfo_Suárez_Madrid–Baraja...,L'aeroporto Adolfo Suarez Madrid-Barajas è ges...


In [20]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="torch.utils.checkpoint")
warnings.filterwarnings("ignore", category=UserWarning, message="None of the inputs have requires_grad=True")

In [29]:
triples = []
predictions = []
actuals = []

i = 0
for tripla in X_test:
  if i < 1010:
    actual = y_test[i]
    prediction = generate_text(tripla, actual)

    triples.append(tripla)
    actuals.append(actual)
    predictions.append(prediction)

    print(f'======== STEP {i} ======== ')
    print(f'Tripla: {tripla}')
    print(f'Actual: {actual}')
    print(f'Prediction: {prediction}')
    print('\n')

    df = pd.DataFrame(list(zip(triples, predictions, actuals)), columns =['triples', 'predictions', 'actuals'])
    df.to_csv(f'{data_path_models}/{new_model}_decoding.csv', index=False)

  i = i + 1


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Output streaming troncato alle ultime 5000 righe.


======== STEP 177 ======== 
Tripla: Bootleg_Series_Volume_1:_The_Quine_Tapes precededBy Squeeze_(The_Velvet_Underground_album) Bootleg_Series_Volume_1:_The_Quine_Tapes recordLabel Polydor_Records Bootleg_Series_Volume_1:_The_Quine_Tapes recordLabel Universal_Music_Group Bootleg_Series_Volume_1:_The_Quine_Tapes recordedIn United_States Bootleg_Series_Volume_1:_The_Quine_Tapes recordedIn St._Louis,_Missouri Bootleg_Series_Volume_1:_The_Quine_Tapes releaseDate 2001-10-16 Bootleg_Series_Volume_1:_The_Quine_Tapes runtime 230.05 
Actual: Dopo aver pubblicato l'album Squeeze, il gruppo musicale Velvet Underground ha prodotto e pubblicato l'album di musica rock Bootleg Series Volume 1: The Quine Tapes, registrato a St Louis Missouri, USA L'album è stato pubblicato il 2001-10-16 sotto le etichette discografiche Polydor e Universal Music Group e ha una durata totale di 230:05 minuti.
Prediction: Bootleg Series Vol.1: The Quine Tapes è stato pre